In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import utils
from tensorflow.keras.datasets import mnist
import seaborn as sns
from tensorflow.keras.initializers import RandomNormal
import time
import matplotlib.pyplot as plt 

In [2]:
#Function for plots for each epoch and error
def plt_dynamic(x, vy, ty, ax, colors= ['b'] ):
    ax.plot(x, vy, 'b', label = "Validation Loss")
    ax.plot(x, ty, 'r', label = "Train Loss")
    plt.legend()
    plt.grid()
    fig.canvas.draw() 

In [3]:
(X_train, y_train),(X_test , y_test) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [4]:

X_train = X_train.reshape(X_train.shape[0], X_train.shape[1]*X_train.shape[2]) 
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1]*X_test.shape[2]) 

In [5]:
#As data is not normalized , let's normalize the data first
X_train = X_train/255
X_test = X_test/255

In [6]:
#As the output is not categorical , let's make it categorical 
Y_train = utils.to_categorical(y_train, 10)
Y_test = utils.to_categorical(y_test, 10)

In [7]:
#Sequential : Linear stack of Layers
#Dense : it will do y = activation(WT. X + b)
#Activation : It is for activation function
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.layers import Dropout
from keras.layers.normalization import BatchNormalization
from tensorflow.keras.optimizers import Adam,RMSprop,SGD

In [8]:
#Some model Parameters
output_dim = 10
input_dim = X_train.shape[1]
batch_size = 128
nb_epoch = 20

In [9]:
def best_hyperparameters(activ):

    model = Sequential()
    model.add(Dense(512, activation=activ, input_shape=(input_dim,), kernel_initializer=RandomNormal(mean=0.0, stddev=0.062, seed=None)))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(128, activation=activ, kernel_initializer=RandomNormal(mean=0.0, stddev=0.125, seed=None)) )
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(output_dim, activation='softmax'))


    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
    
    return model

In [10]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

activ = ['sigmoid','relu']
model = KerasClassifier(build_fn=best_hyperparameters, epochs=nb_epoch, batch_size=batch_size, verbose=0)
param_grid = dict(activ=activ)

grid = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result = grid.fit(X_train, Y_train)

In [11]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.980167 using {'activ': 'relu'}
0.975100 (0.002510) with: {'activ': 'sigmoid'}
0.980167 (0.002181) with: {'activ': 'relu'}
